In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib
import tensorflow as tf
from gmvae import GMVAE

# ==========================
# Load and preprocess the NSL-KDD dataset
# ==========================
data = pd.read_csv('./dataset/kdd_full_clean_5classes.csv')
print('Original shape:', data.shape)

X = data.drop(columns=['target'])
y = data['target']

# Apply fitted transformer
tdt = joblib.load("./typed_nslkdd_all_features.pkl")
X_encoded = tdt.transform(data)

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.3, random_state=42, stratify=y
)

# Encode labels
encoder_label = LabelEncoder()
y_train_encoded = encoder_label.fit_transform(y_train)

# ==========================
# Training parameters
# ==========================
latent_dim = 128
batch_size = 128
epochs = 30
lambda_kl = 0.05
input_dim = x_train.shape[1]
n_classes = len(np.unique(y_train_encoded))

# ==========================
# Prepare GMVAE and dataset
# ==========================
model = GMVAE(
    input_dim=input_dim,
    latent_dim=latent_dim,
    n_classes=n_classes,
    lambda_kl=lambda_kl
)

# TensorFlow dataset
X_tf = tf.convert_to_tensor(x_train, dtype=tf.float32)
Y_tf = tf.convert_to_tensor(y_train_encoded, dtype=tf.int32)
train_dataset = tf.data.Dataset.from_tensor_slices((X_tf, Y_tf)).shuffle(1000).batch(batch_size)

# ==========================
# Train the model
# ==========================
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.train(train_dataset, optimizer=optimizer, epochs=epochs)

# Trigger a dummy forward pass to ensure the model is 'built'
_ = model(tf.convert_to_tensor(x_train[:1], dtype=tf.float32))

# ==========================
# Save model and learned parameters
# ==========================
model.save_weights("./models_training/gmvae_weights.weights.h5")


2025-04-16 12:12:33.198717: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-16 12:12:33.213337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744798353.229667   84882 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744798353.234586   84882 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744798353.247742   84882 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Original shape: (148517, 42)


I0000 00:00:1744798358.059028   84882 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1768 MB memory:  -> device: 0, name: NVIDIA RTX A1000 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6



Epoch 1/30


Batch:  99%|█████████▉| 808/813 [00:18<00:00, 55.85it/s, Loss=3.99, Recon=3.58, Class=0.05, KL=7.05]      2025-04-16 12:12:56.810432: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

→ Epoch 26 | Total Loss: 17427.9 | Recon: 15919.8 | Class: 548.6 | KL: 19192.1

Epoch 27/30


→ Epoch 27 | Total Loss: 17162.4 | Recon: 15918.4 | Class: 444.3 | KL: 15995.4

Epoch 28/30


→ Epoch 28 | Total Loss: 17116.7 | Recon: 15918.7 | Class: 435.0 | KL: 15260.7

Epoch 29/30


→ Epoch 29 | Total Loss: 17397.0 | Recon: 15916.7 | Class: 537.5 | KL: 18855.7

Epoch 30/30


→ Epoch 30 | Total Loss: 17203.8 | Recon: 15917.9 | Class: 461.8 | KL: 16483.3
